In [ ]:
from IPython.utils import io

with io.capture_output() as captured:
  !pip install folium
print("Installation 1 Complete")

with io.capture_output() as captured:
  !pip install openrouteservice
print("Installation 2 Complete")

Installation 1 Complete
Installation 2 Complete


In [ ]:
import openrouteservice as ors
import folium
import numpy as np
import pandas as pd

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
geocoder  = Nominatim(user_agent="My App")
import re

# Adding 1 second padding between calls to server
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geocoder.geocode, min_delay_seconds = 1,   return_value_on_exception = None) 

#Connect to Google Sheets
from gspread import authorize
from gspread.models import Cell
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import drive
drive.mount("/content/drive")
scopes = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]

cred = ServiceAccountCredentials.from_json_keyfile_name("https://drive.google.com/file/d/1Fhd5sEgdGdHVIq9Di0jncDMBa1pOJFBD/view?usp=sharing", scopes)
gclient = authorize(cred)

# Define lat and long functions
def find_loc(address):
  address = re.sub(" Ste \d+", "", address) #remove suite 
  #address = re.sub(",.*,", "", address) #remove city
  if (address == 'Missing'):
    location = 'No address input'   
  else:    
    try: location = geocoder.geocode(address)
    except GeocoderTimedOut as e: location = "Timeout Error"
  if location is None: 
    location = 'Not found by geopy'   
  return location

def coords(key):
    return (find_long[key], find_lat[key])

def find_lat(address):
  location = find_loc(address)
  if isinstance(location, str): return location
  else: return round(location.latitude,6)

def find_long(address):
  location = find_loc(address)
  if isinstance(location, str): return location
  else: return round(location.longitude,6)


Mounted at /content/drive


FileNotFoundError: ignored

In [ ]:
# Add Coordinates for New Locations (~1 sec / new location)

# Access Google Sheet
worksheet = gclient.open("BOTM Database").sheet1
rows = worksheet.get_all_values()

# Convert to a DataFrame
columns = rows[0]
df = pd.DataFrame.from_records(rows[1:], columns=columns) 
new_df = df[(df['LAT'] == '') | (df['LONG'] == '')]

# Calc missing coordinates
cell_list = []
lat_col = df.columns.get_loc('LAT') + 1
long_col = df.columns.get_loc('LONG') + 1
for index, row in new_df.iterrows():
  #df['LAT'] = df.apply(lambda row: find_lat(row['ADDRESS']) if row['LAT'] == '' else row['LAT'], axis=1)
  cell_list.append(Cell(row=index+2, col=lat_col, value=find_lat(row['ADDRESS'])))
  cell_list.append(Cell(row=index+2, col=long_col, value=find_long(row['ADDRESS'])))

if cell_list != []: worksheet.update_cells(cell_list)

In [ ]:
from os import truncate
from folium.plugins import BeautifyIcon


# Access Google Sheet
worksheet = gclient.open("BOTM Database").sheet1
rows = worksheet.get_all_values()
columns = rows[0]
df = pd.DataFrame.from_records(rows[1:20], columns=columns) 

#filter by numerical lat/long and day of the week
df = df[df['LAT'].apply(lambda x: not re.search("[A-Za-z]", x))]
df = df[df['LONG'].apply(lambda x: not re.search("[A-Za-z]", x))]
df = df[(df['LAT'] != '') | (df['LONG'] != '')]

df_mon = df[df['MON']=="1"]

from pandas._libs.hashtable import value_count
    
# The vehicles are all located at BOTM
depot = [29.660709,-82.328536]

# Define the vehicles
# https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Vehicle
vehicles = list()
for idx in range(3): #Number of vehicles
    vehicles.append(
        ors.optimization.Vehicle(
            id=idx, 
            start=list(reversed(depot)),
            end=list(reversed(depot)),
            capacity=[300],
            time_window=[1553241600, 1553284800]  # Fri 8-20:00, expressed in POSIX timestamp
        )
    )
    
# Next define the delivery stations
# # https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Job
deliveries = list()
for delivery in df_mon.itertuples():
    loc_long = float(delivery.LONG)
    loc_lat = float(delivery.LAT)
    deliveries.append(
        ors.optimization.Job(
            id=delivery.Index,
            location=[loc_long, loc_lat], #REVERSED
            service= 20*60,  # Assume 20 minutes at each site
            amount = [50],  
            #amount= int(delivery.Amount),
            #time_windows=[[
                #int(delivery.Open_From.timestamp()),  # VROOM expects UNIX timestamp
                #int(delivery.Open_To.timestamp())
            #]]
      ))

In [ ]:
# Define the map centered around BOTM
f = folium.Figure(height=500, width=1000)
m = folium.Map(location=[29.616321, -82.422549], zoom_start=10).add_to(f)

folium.Marker(
        location=depot,
        icon=folium.Icon(color="green", icon="bus", prefix='fa')#,
        #setZIndexOffset=1000
        ).add_to(m)

# Plot the locations on the map with more info in the ToolTip
for index, row in df_mon.iterrows():
    lat = float(row['LAT'])
    long = float(row['LONG'])

    tooltip = (folium.Tooltip(
        """<h5>{LOCATION} / {NOTES} </h5>
        <b>Address:</b> {ADDRESS} <br>
        <b>Location:</b> ({LAT}, {LONG}) <br>
        """.format(**row)))
    
    folium.Marker(
        location=(lat, long),
        tooltip=tooltip
        #popup=row['LOCATION']
        ).add_to(m)

    # icon=BeautifyIcon(
    #             icon_shape='marker',
    #             number=index, #int(location.Index)
    #             spin=True,
    #             text_color='red',
    #             background_color="#FFF",
    #             inner_icon_style="font-size:12px;padding-top:-5px;"
    #             ).add_to(m)

# Initialize a client and make the request
ors_client = ors.Client(key='5b3ce3597851110001cf62489217d34c333d469aa2034e367d70cb0a')  # Get an API key from https://openrouteservice.org/dev/#/signup
result = ors_client.optimization(
    jobs=deliveries,
    vehicles=vehicles,
    geometry=True,
)

# Add the output to the map
for color, route in zip(['green', 'red', 'blue'], result['routes']):
    decoded = ors.convert.decode_polyline(route['geometry'])  # Route geometry is encoded
    gj = folium.GeoJson(
        name='Vehicle {}'.format(route['vehicle']),
        data={"type": "FeatureCollection", "features": [{"type": "Feature",
                                                         "geometry": decoded,
                                                         "properties": {"color": color}
                                                         }]},
        style_function=lambda x: {"color": x['properties']['color']}
    )
    
    route["miles"] = round(route["distance"] / 1000 * 0.621371, 1)
    route["hrs"] = route["duration"] // 3600
    route["mins"] = round((route["duration"] % 3600) / 60)

    gj.add_child(folium.Tooltip(
        """<h5>Vehicle {vehicle}</h5>
        <b>Distance:</b> {miles} mi <br>
        <b>Duration:</b> {hrs} hr {mins} min
        """.format(**route)
    ))
    gj.add_to(m)

folium.LayerControl().add_to(m)
f